# Liftover from hg19(GRCh37) to hg38 

The pipeline is inspired by [https://www.biostars.org/p/252938/](https://www.biostars.org/p/252938/)

Download required tools: `liftOver`, `liftOverPlink.py` and `hg19ToHg38.over.chain.gz` by
```
wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/liftOver
wget http://hgdownload.soe.ucsc.edu/goldenPath/hg19/liftOver/hg19ToHg38.over.chain.gz
wget https://raw.githubusercontent.com/Shicheng-Guo/GscPythonUtility/master/liftOverPlink.py
```

The python version of liftover document: https://github.com/sritchie73/liftOverPlink

### An error found during the process

A similar one online: https://www.biostars.org/p/369469/

In [3]:
sos run liftover.ipynb -h

usage: sos run liftover.ipynb [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  liftover

Global Workflow Options:
  --numThreads 2 (as int)
                        Specific number of threads to use
  --job-size 1 (as int)
                        For cluster jobs, number commands to run per job
  --walltime 36h
                        Wall clock time expected
  --mem 30G
                        Memory expected
  --container-lmm 'statisticalgenetics/lmm:2.4'
                        Plink provider

Sections
  liftover:             Merge all the *.bim files into a single file. Needs to
                        be run once per type of data (e.g. genotype, exome)
    Workflow Options:
      -

In [27]:
sos run liftover.ipynb liftover\
    --bfiles ~/project/guangyou/data/pops-MWE/1000G.EUR.mini.bed\
    --liftover /mnt/mfs/statgen/guangyou/liftover\
    --chain /mnt/mfs/statgen/guangyou/liftover/hg19ToHg38.over.chain.gz\
    --out /mnt/mfs/statgen/guangyou/liftover/MWE/mini.hg37\
    --container_lmm /mnt/mfs/statgen/containers/lmm.sif

INFO: Running liftover: Merge all the *.bim files into a single file. Needs to be run once per type of data (e.g. genotype, exome)
INFO: liftover is completed.
INFO: liftover output:   /mnt/mfs/statgen/guangyou/liftover/MWE/mini.hg37.ped /mnt/mfs/statgen/guangyou/liftover/MWE/mini.hg37.map
INFO: Workflow liftover (ID=we3ade29951c120e1) is executed successfully with 1 completed step.



In [ ]:
[global]
# Specific number of threads to use
parameter: numThreads = 2
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "36h"
# Memory expected
parameter: mem = "30G"
# Plink provider
parameter: container_lmm = 'statisticalgenetics/lmm:2.4'

In [ ]:
# Merge all the *.bim files into a single file. Needs to be run once per type of data (e.g. genotype, exome)
[liftover]
# Path of liftover program 
parameter: liftover = str
# Path of the chain file
parameter: chain = path
# Path to the *.bim/bed/fam files to liftover
parameter: bfiles = paths
# Output prefix
parameter: out = path
input: bfiles
output: f"{out}.ped",f"{out}.map"
task: trunk_workers = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: expand= "${ }", stderr = f'{step_name}.stderr', stdout = f'{step_name}.stdout' 
    # bed to ped
    plink --bfile  ${_input:n} --recode --out ${out}
    # liftover
    ${liftover}/liftOverPlink.py -m ${_output[1]} -p ${_output[0]} -o ${_output[0]:d}/hgdp_hg38 -c ${chain} -e ${liftover}/liftOver
    # ped to bed
    plink --file ${_output[0]:d}/hgdp_hg38 --make-bed  --allow-extra-chr --chr 1-22 --out ${_output[0]:d}/hgdp_hg38